This notebook implements intersection of convex polygons based on my understanding of this paper:

https://doi.org/10.1016/0146-664X(82)90023-5

with a PDF accessible from:

https://www.cs.jhu.edu/~misha/Spring16/ORourke82.pdf

In [ ]:
import numpy as np
from tweakwcs.linearfit import build_fit_matrix

In [ ]:
class Vertex:
    def __init__(self, x, y):
        self._xy = [x, y]
        
    def __repr__(self):
        return f"<{self.x:.3g}, {self.y:.3g}>"
    
    @property
    def x(self):
        return self._xy[0]
    
    @x.setter
    def x(self, x):
        self._xy[0] = x
    
    @property
    def y(self):
        return self._xy[1]
    
    @y.setter
    def y(self, y):
        self._xy[1] = y
        
    def __iter__(self):
        for i in range(2):
            yield self._xy[i]
    
    def __getitem__(self, key):
        return self._xy[key]
    
    def __setitem__(self, key, value):
        self._xy[key] = value
    
    def __add__(self, other):
        return Vertex(self.x + other[0], self.y + other[1])
    
    def __iadd__(self, other):
        self._xy[0] += other[0]
        self._xy[1] += other[1]
        return self
    
    def __sub__(self, other):
        return Vertex(self.x - other[0], self.y - other[1])
    
    def __isub__(self, other):
        self._xy[0] -= other[0]
        self._xy[1] -= other[1]
        return self
    
    def __mul__(self, other):
        return Vertex(self.x * other, self.y * other)
    
    def __imul__(self, other):
        self._xy[0] *= other
        self._xy[1] *= other
        return self
    
    def dot(self, other):
        return self.x * other[0] + self.y * other[1]
    
    def cross(self, other):
        return self.x * other[1] - self.y * other[0]


def subtended(p, v1, v2):
    v1 = v1 - p
    v2 = v2 - p
    dot = v1.dot(v2)
    cross = v1.cross(v2)
    return np.rad2deg(np.arctan2(cross, dot))


def is_hp(p_, p, pt):
    return ((p - p_).cross(pt - p_) >= 0)


def make_ccw_poly(p, debug=True):
    # direction of transversing a polygon
    c = np.mean(list(map(list, p)), axis=0)
    if is_hp(p[0], p[1], Vertex(*c)):
        return p
    else:
        if debug:
            print("Reversing polygon direction (P)")
        return p[::-1]


def intersect(p, q, debug=True):
    if np.allclose(list(p[0]), list(p[-1]), rtol=1e-8):
        if debug:
            print("Removing endpoint vertex. polygon must be open")
            
    p = make_ccw_poly(p, debug)
    q = make_ccw_poly(q, debug)
    
    lenp = len(p)
    lenq = len(q)
    assert lenp > 2
    assert lenq > 2
    
    if debug:
        print(f"Input polygon P: {p}")
        print(f"Input polygon Q: {q}")

    ip = 0
    iq = 0

    first_intersect = None

    # output polygon
    poly = []
    inside = None #"P"

    pv_ = p[(ip - 1) % lenp]
    pv = p[ip % lenp]
    qv_ = q[(iq - 1) % lenq]
    qv = q[iq % lenq]

    first_k = -2
    for k in range(2 * (lenp + lenq)):
        output_vertex = None
        if debug:
            print(f"\n*** k: {k}  ip: {ip}  iq: {iq}")
            print(f"p: {pv}  p_: {pv_}")
            print(f"q: {qv}  q_: {qv_}")

        dp = pv - pv_
        dq = qv - qv_

        # https://en.wikipedia.org/wiki/Line–line_intersection
        t = (pv_.x - qv_.x) * (qv_.y - qv.y) - (pv_.y - qv_.y) * (qv_.x - qv.x)
        u = (pv_.x - qv_.x) * (pv_.y - pv.y) - (pv_.y - qv_.y) * (pv_.x - pv.x)
        # d = (pv_.x - pv.x) * (qv_.y - qv.y) - (pv_.x - pv.y) * (qv_.x - qv.x)
        cross = dp.cross(dq)
        if cross < 0:
            t = -t
            u = -u
            d = -cross
        else:
            d = cross

        if debug:
            print(f"Intersection check: t={t:.5e},  u={u:.5e},  d={d:.5e}")

        if ((0 <= t <= d) and (0 <= u <= d)) and d > 1e-6:
            if debug:
                print("Intersection detected between p and q")
            t = t / d
            u = u / d
            xi = pv_.x + (pv.x - pv_.x) * t
            yi = pv_.y + (pv.y - pv_.y) * t
            output_vertex = Vertex(xi, yi)
            if debug:
                print(f"Intersection point (xi, yi): {output_vertex}")

            if first_intersect is None:
                if debug:
                    print("* first intersection!")
                first_intersect = output_vertex
                first_k = k
            #elif abs(first_intersection[0] - xi) < 1e-6 and abs(first_intersection[1] - yi) < 1e-6:
            elif k - first_k != 1 and abs(first_intersect.x - xi) == 0 and abs(first_intersect.y - yi) == 0:
                if debug:
                    print(f"* END: break loop - same intersection  new: "
                          f"{output_vertex}   -- first: {first_intersect}")
                break

            inside = "Q" if is_hp(qv_, qv, pv) else "P"
            if debug:
                print(f"Setting 'INSIDE': {repr(inside)}")

        if dq.cross(dp) >= 0.0:
            if debug:
                print("* ADVANCING - branch 1")

            if is_hp(qv_, qv, pv):
                if inside == "Q" and not output_vertex:
                    if debug:
                        print(f"Appending (qx, qy): {qv}")
                    poly.append(qv)
                iq += 1
                qv_ = qv
                qv = q[iq % lenq]
                if debug:
                    print("* advancing Q - branch 1: "
                          f"iq={iq}, {iq} mod {lenq}={iq % lenq}, qv_: {qv_},  qv: {qv}")
            else:
                if inside == "P" and not output_vertex:
                    if debug:
                        print(f"Appending (px, py): {pv}")
                    poly.append(pv)
                ip += 1
                pv_ = pv
                pv = p[ip % lenp]
                if debug:
                    print("* advancing P - branch 1: "
                          f"ip={ip}, {ip} mod {lenp}={ip % lenp}, pv_: {pv_},  pv: {pv}")

        else:
            print("* ADVANCING - branch 2")

            if is_hp(pv_, pv, qv):
                if inside == "P" and not output_vertex:
                    if debug:
                        print(f"Appending (px, py): {pv}")
                    poly.append(pv)
                ip += 1
                pv_ = pv
                pv = p[ip % lenp]
                if debug:
                    print("* advancing P - branch 2: "
                          f"ip={ip}, {ip} mod {lenp}={ip % lenp}, pv_: {pv_},  pv: {pv}")

            else:
                if inside == "Q" and not output_vertex:
                    if debug:
                        print(f"Appending (qx, qy): {qv}")
                    poly.append(qv)
                iq += 1
                qv_ = qv
                qv = q[iq % lenq]
                if debug:
                    print("* advancing Q - branch 2: "
                          f"iq={iq}, {iq} mod {lenq}={iq % lenq}, qv_: {qv_},  qv: {qv}")
                
    return poly

In [ ]:
p = [(0, 0), (1, 0), (1, 1), (0, 1)]
# p = [(1, 1), (0, 1), (0, 0), (1, 0)][::-1]  # To test CW->CCW conversion

# test 1
q = [(x + 0.25, y + 0.1) for x, y in p]

# # test 2:
# R = build_fit_matrix(45)
# q = [tuple(np.dot(R, pk).tolist()) for pk in p]

p = [Vertex(*v) for v in p]
q = [Vertex(*v) for v in q]

intersect(p, q)